In [2]:
import pandas as pd
import numpy as np

/Users/asadsheikh/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score,precision_score, recall_score


In [4]:
df = pd.read_csv('crime_data.csv')

/Users/asadsheikh/.pyenv/versions/3.8.0/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
df_filtered = df.drop(columns = ['ID', 'Case Number', 'Date', 'Description', 'Location Description', 'X Coordinate', 'Y Coordinate', 'Updated On', 'Latitude', 'Longitude', 'Location'])
df_filtered = df_filtered.dropna(axis=0)

In [20]:
df_filtered = df_filtered[df_filtered['Year']==2020]
df = df_filtered

In [21]:
X = df_filtered.drop(columns=['Arrest'])
df['Beat'] = df['Beat'].astype('category')
df['District'] = df['District'].astype('category')
df['Ward'] = df['Ward'].astype('category')
df['Community Area'] = df['Community Area'].astype('category')
df['Block'] = df['Block'].astype('category')
df['IUCR'] = df['IUCR'].astype('category')
df['Primary Type'] = df['Primary Type'].astype('category')

In [22]:
df.dtypes

Block             category
IUCR              category
Primary Type      category
Arrest                bool
Domestic              bool
Beat              category
District          category
Ward              category
Community Area    category
FBI Code            object
Year                 int64
dtype: object

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['Year']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Block', 'IUCR', 'Primary Type', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier())])

X = df.drop('Arrest', axis=1)
y = df['Arrest']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))


model score: 0.894


In [28]:
models = [LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier(), KNeighborsClassifier(), SGDClassifier()]

In [29]:
cols = []
comp = pd.DataFrame(columns = cols)

for i in range(len(models)):
    alg = models[i]
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', alg)])
    predicted = clf.fit(X_train, y_train).predict(X_test)
    name = alg.__class__.__name__
    comp.loc[i,'Model Name'] = MLA_name
    comp.loc[i, 'Train Accuracy'] = round(clf.score(X_train, y_train), 2)
    comp.loc[i, 'Test Accuracy'] = round(clf.score(X_test, y_test), 2)
    comp.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)  
    
MLA_compare

/Users/asadsheikh/.pyenv/versions/3.8.0/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model Name,Train Accuracy,Test Accuracy,Precision,Recall,F1 score
0,LogisticRegression,0.92,0.91,0.88,0.52,0.65
4,SGDClassifier,0.91,0.91,0.88,0.51,0.65
2,RandomForestClassifier,0.97,0.90,0.79,0.54,0.64
1,DecisionTreeClassifier,0.97,0.89,0.74,0.56,0.63
3,KNeighborsClassifier,0.91,0.89,0.78,0.49,0.61


Block               object
IUCR                object
Primary Type        object
Domestic              bool
Beat              category
District          category
Ward              category
Community Area    category
FBI Code            object
Year                 int64
dtype: object

/Users/asadsheikh/.pyenv/versions/3.8.0/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KeyboardInterrupt: 